In [1]:
from models.ResNet_3D.model import generate_model
from models.ResNet_3D.opts import create_args
from models.ResNet_3D.mean import get_mean, get_std

import torch

In [2]:
opt = create_args(model_name = 'resnext', model_depth=101, \
                  model='/home/nevin/nevin/weights/3D-resnet/resnext-101-kinetics.pth', \
                  no_cuda=False)
opt.mean = get_mean()
opt.std = get_std()
opt.arch = '{}-{}'.format(opt.model_name, opt.model_depth)
opt.sample_size = 112
opt.sample_duration = 16
opt.n_classes = 400

In [4]:
model = generate_model(opt)
print('loading model {}'.format(opt.model))
if(opt.no_cuda):
    model_data = torch.load(opt.model, map_location='cpu')
else:
    model_data = torch.load(opt.model)
assert opt.arch == model_data['arch']
model.load_state_dict(model_data['state_dict'],strict=False)
model = model.eval()

RuntimeError: Cannot initialize CUDA without ATen_cuda library. PyTorch splits its backend into two shared libraries: a CPU library and a CUDA library; this error has occurred because you are trying to use some CUDA functionality, but the CUDA library has not been loaded by the dynamic linker for some reason.  The CUDA library MUST be loaded, EVEN IF you don't directly use any symbols from the CUDA library! One common culprit is a lack of -Wl,--no-as-needed in your link arguments; many dynamic linkers will delete dynamic library dependencies if you don't depend on any of their symbols.  You can check if this has occurred by using ldd on your binary to see if there is a dependency on *_cuda.so library.

In [3]:
def find_length(filename, seg_length=1):
    """
    Function to find the length of the video in terms of specified segments.
    """
    return sum(1 for _ in generate_block(filename, 1))

In [4]:
def generate_resnet_features(net, filename):
    """
    Function to process the current video file and generate the resnet fetures.
    
    Parameters
    net             :torch.nn.Module
                     The resnet feature extraction model.
    filename        :str
                     The filename of the videos to return the features
    
    Returns
    -------
    features        :numpy.array
                     The features of the video in the required shape.
    """
    total_length = find_length(filename)
    block = generate_block(filename, 1)
    start_time = time.time()
    feature_arr = []
    for i, curr_block in enumerate(block):
        print("{}/{}".format(i+1, total_length), sep='\r', end='\r')
        features = net(curr_block)
        features = (features - features.mean())/(features.max() - features.mean())
        feature_arr.append(features.detach().numpy())
        del features
    print("")
    total_time = time.time()-start_time
    features = np.rollaxis(np.array(feature_arr), 1 )[0]
    return features

In [38]:
from torchvision.transforms import Normalize, Compose, Resize, ToPILImage, ToTensor

In [16]:
import numpy as np

In [66]:
img = cv2.imread('/home/nevin/Pictures/Screenshot from 2019-04-12 19-54-41.png')

In [31]:
arr = []
for i in range(16):
    arr.append(img)

In [53]:
transforms = Compose([\
                          ToPILImage(),\
                          Resize((112, 112), interpolation=2),\
                          ToTensor(),\
                          Normalize(mean=opt.mean, std=opt.std)\
                     ])

In [62]:
tens_array = [transforms(img).view(1, 3, 112, 112) for img in arr]

In [63]:
torch.cat(tens_array, dim=0).shape

torch.Size([16, 3, 112, 112])

In [45]:
tens_array[0].shape

torch.Size([3, 112, 112])

In [54]:
ss = transforms(img)

In [55]:
ss.shape

torch.Size([3, 112, 112])

In [67]:
img.shape

(768, 1366, 3)

In [68]:
ss.dtype

torch.float32